In [ ]:
!pip install -q sentence-transformers faiss-cpu langchain-text-splitters
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
import requests
from google.colab import files
import os

uploaded = files.upload()
file_path = next(iter(uploaded))
ext = os.path.splitext(file_path)[1]
text = ""
if ext == ".txt":
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
elif ext == ".pdf":
    !pip install -q PyMuPDF
    import fitz
    doc = fitz.open(file_path)
    text = "\n".join(page.get_text() for page in doc)
else:
    raise ValueError("Unsupported file type. Please upload a .txt or .pdf file.")

text = text[:20000]

Saving AI_in_Education_Research_Article.pdf to AI_in_Education_Research_Article.pdf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 22.4 MB/s eta 0:00:00


In [ ]:
def fixed_size_chunk(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

from langchain_text_splitters import RecursiveCharacterTextSplitter
def recursive_chunk(text, chunk_size=1000, overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
    )
    return [chunk.page_content for chunk in splitter.create_documents([text])]

In [ ]:
chunks_fixed = fixed_size_chunk(text)
chunks_recursive = recursive_chunk(text)

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings_fixed = model.encode(chunks_fixed, show_progress_bar=True)
embeddings_recursive = model.encode(chunks_recursive, show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import faiss
import numpy as np

dim = embeddings_fixed.shape[1]
index_fixed = faiss.IndexFlatL2(dim)
index_recursive = faiss.IndexFlatL2(dim)

index_fixed.add(np.array(embeddings_fixed))
index_recursive.add(np.array(embeddings_recursive))

In [ ]:
queries = [
    "What is the title of the research article?",
    "What is one of the applications of AI in classrooms mentioned in the abstract?",
    "How can AI help with student engagement outside classroom hours?",
    "What is a concern regarding the use of AI for automated assessment and grading?",
    "What do AI-powered adaptive learning systems tailor educational content to?"
]
def search_index(query, index, chunks, top_k=1):
    embedding = model.encode([query])
    D, I = index.search(np.array(embedding), top_k)
    return [chunks[i] for i in I[0]]

In [ ]:
for q in queries:
    print(f"\n-------------------------------------------------Query: {q}----------------------------------------------------------------------------")
    print("===============Fixed Chunk Response:")
    print(search_index(q, index_fixed, chunks_fixed)[0][:500], "\n")
    print("===============Recursive Chunk Response:")
    print(search_index(q, index_recursive, chunks_recursive)[0][:500])


-------------------------------------------------Query: What is the title of the research article?----------------------------------------------------------------------------
===============Fixed Chunk Response:
 (2021). AI and Education: Guidance for Policy-makers.
[3] Luckin, R., Holmes, W., Griffiths, M., & Forcier, L. B. (2016). Intelligence Unleashed: An
argument for AI in Education.
 

===============Recursive Chunk Response:
Section 2: Intelligent Tutoring Systems
Intelligent tutoring systems (ITS) use AI algorithms to provide immediate and personalized feedback
to students. ITS simulates a one-on-one teaching environment, improving learning efficacy and
engagement.
References:
[1] Holmes, W., Bialik, M., & Fadel, C. (2019). Artificial Intelligence in Education: Promises and
Implications for Teaching and Learning.
[2] UNESCO. (2021). AI and Education: Guidance for Policy-makers.
[3] Luckin, R., Holmes, W., Griffiths

-------------------------------------------------Query: What 

In [ ]:
# Part 2

In [ ]:
ground_truths = [
    "The title of the research article is - The Impact of Artificial Intelligence on Modern Education.",
    "One application of AI in classrooms mentioned in the abstract is automated grading.",
    " AI tools like Natural Language Processing (NLP) and AI chatbots can answer common questions, provide resources, and support learning outside classroom hours.",
    "A concern regarding the use of AI for automated assessment and grading is fairness and transparency.",
    "AI-powered adaptive learning systems tailor educational content to individual student needs, learning paces, and preferences."
]
def base_prompt(context, query):
    return f"""Context:
{context}

Question: {query}
Answer:"""

def react_prompt(context, query):
    return f"""You are a thoughtful assistant. Think step by step before answering.

Context:
{context}

Question: {query}
Let's think carefully:"""

def dsp_prompt(context, query):
    return f"""Please focus on only the key details from the passage.

Context:
{context}

Question: {query}
Important detail:"""

In [ ]:
from groq import Groq
import os

os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ["GROQ_API_KEY"])

def query_llama3(prompt):
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=100,
        top_p=1,
        stop=None,
    )
    return response.choices[0].message.content.strip()

In [ ]:
from sklearn.metrics import f1_score
import re

def clean_tokens(t):
    return re.sub(r"[^a-zA-Z ]", "", t.lower()).split()

def compute_f1s(preds, targets):
    scores = []
    for pred, true in zip(preds, targets):
        y_pred = clean_tokens(pred)
        y_true = clean_tokens(true)
        common = set(y_pred) & set(y_true)
        if len(y_pred) + len(y_true) == 0:
            scores.append(1.0)
        elif len(common) == 0:
            scores.append(0.0)
        else:
            precision = len(common) / len(y_pred)
            recall = len(common) / len(y_true)
            f1 = 2 * (precision * recall) / (precision + recall)
            scores.append(f1)
    return scores

In [ ]:
import pandas as pd

def run_rag_test(index, chunks, prompt_func):
    predictions = []
    for q in queries:
        context = search_index(q, index, chunks)[0]
        prompt = prompt_func(context, q)
        answer = query_llama3(prompt)
        predictions.append(answer)
    return predictions

# Run for each prompting method
base_preds = run_rag_test(index_fixed, chunks_fixed, base_prompt)
react_preds = run_rag_test(index_fixed, chunks_fixed, react_prompt)
dsp_preds = run_rag_test(index_fixed, chunks_fixed, dsp_prompt)

# F1 scores
results = {
    "Base Prompt": compute_f1s(base_preds, ground_truths),
    "ReAct Prompt": compute_f1s(react_preds, ground_truths),
    "Directional Prompt": compute_f1s(dsp_preds, ground_truths),
}

# Display
df = pd.DataFrame(results, index=[f"Q{i+1}" for i in range(len(queries))])
df["Average"] = df.mean(axis=1)
print(df)
print("\nOverall F1 Averages:")
print(df.mean())

    Base Prompt  ReAct Prompt  Directional Prompt   Average
Q1     0.533333      0.186667            0.500000  0.406667
Q2     0.163265      0.227273            0.327869  0.239469
Q3     0.434783      0.316832            0.418605  0.390073
Q4     0.833333      0.260870            0.833333  0.642512
Q5     0.823529      0.239130            0.838710  0.633790

Overall F1 Averages:
Base Prompt           0.557649
ReAct Prompt          0.246154
Directional Prompt    0.583703
Average               0.462502
dtype: float64
